In [1]:
import time
import re

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
wd = "C:\\Users\\user\\Downloads\\chromedriver_win32\\chromedriver.exe"
addr = "https://news.nate.com/view/20200413n16162?mid=n1008"    # 기사 주소 넣기
addr_nums = re.findall("\d+", addr)
addr_comment = 'http://comm.news.nate.com/Comment/ArticleComment/List?artc_sq='+addr_nums[0]+'n'+addr_nums[1] # 댓글창 주소

In [4]:
driver = webdriver.Chrome(wd)
driver.get(addr_comment)

In [23]:
pages = 1

In [24]:
try:
    for i in range(5):
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"body > div.commentBox.reply_hide > div:nth-child(3) > div.paging_wrap > a:nth-child("+str(pages)+")"))).click()
        time.sleep(2)
        print(pages, end=" ")
        pages+=1
except exceptions.ElementNotVisibleException as e: # 페이지 끝
    pass
except Exception as e: # 다른 예외 발생시 확인
    print(e)

1 2 3 4 5 

In [25]:
html = driver.page_source
dom = BeautifulSoup(html, "lxml")

In [26]:
# 댓글이 들어있는 페이지 전체 크롤링
comments_raw = dom.find_all('dd',{'class':'usertxt'})
# 댓글의 text만 뽑는다.
comments = [comment.text for comment in comments_raw]

In [27]:
comments[:10]

['\n빙시로 아나 ㅋㅋㅋ\t\t\t  ',
 '\n댓글들 기사 읽은 거 맞냐?\n문자 증거 다 있다잖아\n프로포폴 해명 가능하지\n게다가 의사는 구속됨.\n이게 뭘 의미하는지  생각이나 하고 댓글 써라\n하정우는 싹다 고소나 하길\n아까운 젊은이 둘이나 악플들로 보내놓고 \n또 참 잘들 하는 짓이다\t\t\t  ',
 '\n오케이 느그애들 해킹피해금 오케이\n내가 업주들한테 얘기해가\n퇴직금이니 피해금이니 최대한\n챙겨주도록 얘기하께\n근데,느그아들이 여기있는건 안되지\n어디 다른식구끼리 한곳에 있노\nㆍ\nㆍ\n쉐키야 그게 말이가 지금\nㆍ\nㆍ\n쉐키? 임마 나이 먹더니 이제 입에서\n쉐키 소리까지 나오네\n불 함 붙이봐라\nㆍ\nㆍ\n아나~\nㆍ\n붙이봐라\nㆍ\n아따마..우리 최사장 오늘 뭘 잘못드셨나\n행배야 내 이따 아이가 \n어릇을적 니 담뱃불 붙히던 김판호 아이다\nㆍ\nㆍ\n안되겠다 니 오늘 좀 맞자\nㆍ\nㆍ\n쨍그랑 퍽 윽엑 윽\t\t\t  ',
 '\n프로포폴 관련 인터뷰도 해주세요!\t\t\t  ',
 '\n기자 섭외 했네\t\t\t  ',
 '\n아 근데 진짜 실망임ㅠㅠ\n영화에서는 악당들 시원하게 물리치고 사이다 날리는 역할이였으면서ㅠㅠ 현실에서는ㅠㅠ 숨고...물론 이미지라고는 한다해도.\n실망\t\t\t  ',
 '\n이 기사로 유추할 수 있는 것. 1. 가해자들이 협박하면서 형님이라고 했다는 건 상대가 남자들의 공공연한 이상형일 가능성이 높음. 2. 영화개봉에 직접적 영향을 줄 대중적 인기의 상대녀 3. 그렇다면 하정우 입장에서 딱히 이미지 훼손될 이유가 없다고 판단. 평소 하정우는 카톡이나 메시지보다는 차라리 전화하는 편이며, 연인들이랑 연락문제에 취약하다 말하고 다닐 정도라 사생활이 크게 노출이 안됨. 유부남도 아니었고 유명인과의 열애설 정도의 타격이었기에. 4. 협박시기상 전여친이 아닌 현여친일 가능성도 높음.\t\t\t  ',
 '\n피해자욕하지마라  \n몸팔다 당한년들은  피해자니까 욕하지말라면서\t\t\t  ',
 '

In [29]:
normalized_text = []
for string in comments:
    try:
        tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ",string)
    except Exception as e:
        print(string)
        break
    normalized_text.append(tokens)
print(normalized_text[:5])

[' 빙시로 아나 ㅋㅋㅋ ', ' 댓글들 기사 읽은 거 맞냐 문자 증거 다 있다잖아 프로포폴 해명 가능하지 게다가 의사는 구속됨 이게 뭘 의미하는지 생각이나 하고 댓글 써라 하정우는 싹다 고소나 하길 아까운 젊은이 둘이나 악플들로 보내놓고 또 참 잘들 하는 짓이다 ', ' 오케이 느그애들 해킹피해금 오케이 내가 업주들한테 얘기해가 퇴직금이니 피해금이니 최대한 챙겨주도록 얘기하께 근데 느그아들이 여기있는건 안되지 어디 다른식구끼리 한곳에 있노 쉐키야 그게 말이가 지금 쉐키 임마 나이 먹더니 이제 입에서 쉐키 소리까지 나오네 불 함 붙이봐라 아나 붙이봐라 아따마 우리 최사장 오늘 뭘 잘못드셨나 행배야 내 이따 아이가 어릇을적 니 담뱃불 붙히던 김판호 아이다 안되겠다 니 오늘 좀 맞자 쨍그랑 퍽 윽엑 윽 ', ' 프로포폴 관련 인터뷰도 해주세요 ', ' 기자 섭외 했네 ']


In [21]:
p_comment_list = []

In [30]:
for comment in comments:
    p_comment_list.append(normalized_text)

In [34]:
file = open('new2.txt','w',encoding = 'utf-8')
for comment in normalized_text:
    file.write(comment + '\n')
file.close()